In [2]:
from pathlib import Path

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import mne
import mne_connectivity

import acareeg

In [3]:
recompute = False
mastersheet = acareeg.eegip.get_mastersheet()
mastersheet = mastersheet.reset_index()[["Site", "ID", '6m_EEGLAB_QCR_File', 
                                         '12m_EEGLAB_QCR_File', '18m_EEGLAB_QCR_File']]
mastersheet = mastersheet.dropna(how="all")
mastersheet = mastersheet[mastersheet.Site != "Boston"]
mastersheet["subject_no"] = [int(subject[-3:]) if isinstance(subject, str) else subject 
                             for subject in mastersheet["ID"]]

fmin = (3, 6, 9, 30, 4)
fmax = (5, 9, 30, 100, 100)
bands = ("theta", "alpha", "beta", "gamma", "broadband")
con_names = ("ciplv", )
mode = 'multitaper'

common_template_age = 12
use_same_template_age = True
tmax = 1.0
min_nb_epochs = 20
np.random.seed(324234)
con_path = Path("/Volumes/usc_data/ElementSE/eegip/con_paper/")
bids_root = Path("/Volumes/usc_data/ElementSE/eegip")
subjects_dir = "."

In [4]:
"""

for index, row in tqdm(mastersheet.iterrows(), total=len(mastersheet)):
    dataset = row.Site.lower()
    for age in [6, 12, 18]:
        file_name = row[f"{age}m_EEGLAB_QCR_File"]
        epochs = acareeg.eegip.get_resting_state_epochs(row.subject_no, dataset, age, 
                                                        bids_root=bids_root,
                                                        tmax=tmax)
        if epochs is None:
            continue
        if len(epochs) < min_nb_epochs:
            continue

        epochs.set_eeg_reference(projection=True)

        if not isinstance(file_name, str) or "qcr.se" not in file_name:
            # "qcr.se" because .set is currently truncated for Washington
            continue

        if use_same_template_age:
            sources_age = common_template_age
        else:
            sources_age = age            

        # Currently, we don't include the volume sources because there is no way to simulate
        # with mixed source models.                    
        label_ts, anat_label = acareeg.infantmodels.compute_sources(epochs, sources_age, 
                                                                    subjects_dir=subjects_dir, 
                                                                    return_labels=True, return_xr=False, loose=0, 
                                                                    fixed=True, inv_method="eLORETA", pick_ori=None,
                                                                    lambda2=1e-4, minimal_snr=None, verbose=False, 
                                                                    include_vol_src=False)          
        label_names = np.array([label.name for label in anat_label])
        sfreq = epochs.info['sfreq']            

        for con_name in con_names:
            if use_same_template_age:
                path_out = con_path / f"{row.subject_no}_{dataset}_{age}_{con_name}_{common_template_age}m-template.csv"
            else:
                path_out = con_path / f"{row.subject_no}_{dataset}_{age}_{con_name}.csv"
            if path_out.exists() and not recompute:
                continue

            con_df = []       
            con = mne_connectivity.spectral_connectivity_epochs(label_ts, 
                                                                method=con_name,
                                                                mode=mode, sfreq=sfreq, fmin=fmin,
                                                                fmax=fmax, faverage=True, verbose=False)
            dfs = [] 
            for mat, band in zip(con.get_data("dense").transpose(2, 0, 1), bands):
                mat = pd.DataFrame(mat) + np.triu(mat * np.nan)
                mat.index = label_names
                mat.columns = label_names
                df = mat.reset_index().melt(id_vars="index").dropna()
                df.columns = ["region1", "region2", "con"]
                df["con_name"] = con_name
                df["band"] = band
                df["age"] = age
                dfs.append(df)

            pd.concat(dfs).to_csv(path_out)""";

In [5]:
for index, row in tqdm(mastersheet.iterrows(), total=len(mastersheet)):
    dataset = row.Site.lower()
    for age in [6, 12, 18]:

        done = True
        for con_name in con_names:
            if use_same_template_age:
                path_out = con_path / f"{row.subject_no}_{dataset}_{age}_{con_name}_{common_template_age}m-template.csv"
            else:
                path_out = con_path / f"{row.subject_no}_{dataset}_{age}_{con_name}.csv"
            if not path_out.exists():
                done = False

        if done and not recompute:
            print(f"skip {row.subject_no}_{dataset}_{age}")
            continue

        file_name = row[f"{age}m_EEGLAB_QCR_File"]
        epochs = acareeg.eegip.get_resting_state_epochs(row.subject_no, dataset, age, 
                                                        bids_root=bids_root,
                                                        tmax=tmax)

        if epochs is None:
            continue
        if len(epochs) < min_nb_epochs:
            continue

        epochs.set_eeg_reference(projection=True)        

        if not isinstance(file_name, str) or "qcr.se" not in file_name:
            # "qcr.se" because .set is currently truncated for Washington
            continue

        if use_same_template_age:
            sources_age = common_template_age
        else:
            sources_age = age            
   
        
        # Currently, we don't include the volume sources because there is no way to simulate
        # with mixed source models.                    
        label_ts, anat_label = acareeg.infantmodels.compute_sources(epochs, sources_age, 
                                                                    subjects_dir=subjects_dir, 
                                                                    return_labels=True, return_xr=False, loose=0, 
                                                                    fixed=True, inv_method="eLORETA", pick_ori=None,
                                                                    lambda2=1e-4, minimal_snr=None, verbose=False, 
                                                                    include_vol_src=False)        

        label_names = np.array([label.name for label in anat_label])
        sfreq = epochs.info['sfreq']            

        for con_name in con_names:
            if use_same_template_age:
                path_out = con_path / f"{row.subject_no}_{dataset}_{age}_{con_name}_{common_template_age}m-template.csv"
            else:
                path_out = con_path / f"{row.subject_no}_{dataset}_{age}_{con_name}.csv"
            if path_out.exists() and not recompute:
                continue

            nb_iters = int(len(epochs)/min_nb_epochs*2)     
            con_df = []
            for _ in tqdm(list(range(nb_iters)), leave=False):
                inds = np.random.choice(np.arange(len(epochs)), min_nb_epochs, False).astype(int)            
                con = mne_connectivity.spectral_connectivity_epochs(np.array(label_ts)[inds], 
                                                                    method=con_name,
                                                                    mode=mode, sfreq=sfreq, fmin=fmin,
                                                                    fmax=fmax, faverage=True, verbose=False)

                dfs = [] 
                for mat, band in zip(con.get_data("dense").transpose(2, 0, 1), bands):
                    mat = pd.DataFrame(mat) + np.triu(mat * np.nan)
                    mat.index = label_names
                    mat.columns = label_names
                    df = mat.reset_index().melt(id_vars="index").dropna()
                    df.columns = ["region1", "region2", "con"]
                    df["con_name"] = con_name
                    df["band"] = band
                    df["age"] = age
                    dfs.append(df)

                con_df.append(pd.concat(dfs))
            pd.concat(con_df).to_csv(path_out)


  0%|          | 0/192 [00:00<?, ?it/s]

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3301 samples (6.602 s)



/Users/christian/Code/acareeg/acareeg/eegip.py:154: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw = mne.io.read_raw_eeglab(path, preload=True, verbose=verbose)
/Users/christian/Code/acareeg/acareeg/eegip.py:154: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(path, preload=True, verbose=verbose)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 129 out of 129 | elapsed:    0.5s finished
/Users/christian/Code/acareeg/acareeg/eegip.py:95: RuntimeWarning: Estimat

Applying ICA to Raw instance
    Transforming to ICA space (99 components)
    Zeroing out 29 ICA components
    Projecting back using 99 PCA components


/Users/christian/Code/acareeg/acareeg/eegip.py:147: RuntimeWarning: Estimated head radius (10138.0 cm) is above the 99th percentile for adult head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  read_ica_eeglab(file_name).apply(raw, exclude=ind_comp_to_drop)


...


Finished.
Computing rank from data with rank=None


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.5s finished


    Using tolerance 5.4e-10 (2.2e-16 eps * 119 dim * 2.1e+04  max singular value)
    Estimated rank (eeg): 12
    EEG: rank 12 computed from 119 data channels with 1 projector
    Created an SSP operator (subspace dimension = 1)
    Setting small EEG eigenvalues to zero (without PCA)
Reducing data rank from 119 -> 12
Estimating covariance using SHRUNK
Done.
Estimating covariance using DIAGONAL_FIXED
    EEG regularization : 0.1
Done.
Estimating covariance using EMPIRICAL
Done.
Using cross-validation to select the best estimator.
    EEG regularization : 0.1
    EEG regularization : 0.1
    EEG regularization : 0.1
Number of samples used : 102500
log-likelihood on unseen data (descending order):
   shrunk: -52.207
   empirical: -52.208
   diagonal_fixed: -52.885
selecting best estimator: shrunk
[done]
Computing inverse operator with 119 channels.
    119 out of 119 channels remain after picking
Selected 119 channels
Creating the depth weighting matrix...
    119 EEG channels
    limit 

  0%|          | 0/20 [00:00<?, ?it/s]

/var/folders/3k/220tdhsn33709gq9ylr8c6kh0000gp/T/ipykernel_63659/3195365107.py:64: RuntimeWarning: fmin=3.000 Hz corresponds to 3.000 < 5 cycles based on the epoch length 1.000 sec, need at least 1.667 sec epochs or fmin=5.000. Spectrum estimate will be unreliable.
  con = mne_connectivity.spectral_connectivity_epochs(np.array(label_ts)[inds],
/var/folders/3k/220tdhsn33709gq9ylr8c6kh0000gp/T/ipykernel_63659/3195365107.py:64: RuntimeWarning: fmin=3.000 Hz corresponds to 3.000 < 5 cycles based on the epoch length 1.000 sec, need at least 1.667 sec epochs or fmin=5.000. Spectrum estimate will be unreliable.
  con = mne_connectivity.spectral_connectivity_epochs(np.array(label_ts)[inds],
/var/folders/3k/220tdhsn33709gq9ylr8c6kh0000gp/T/ipykernel_63659/3195365107.py:64: RuntimeWarning: fmin=3.000 Hz corresponds to 3.000 < 5 cycles based on the epoch length 1.000 sec, need at least 1.667 sec epochs or fmin=5.000. Spectrum estimate will be unreliable.
  con = mne_connectivity.spectral_connecti